# 5. Customer / Stakeholder Acceptance

## 5.1 Konklusi

Pesatnya pertumbuhan pasar mobil bekas di Arab Saudi, yang didorong oleh meningkatnya kebutuhan kendaraan pribadi dan percepatan digitalisasi melalui platform seperti Syarah.com, menimbulkan tantangan utama dalam penentuan harga mobil yang wajar akibat tingginya variasi kondisi kendaraan. Untuk menjawab tantangan tersebut, proyek ini bertujuan membangun **model machine learning berbasis regresi** yang mampu memprediksi harga wajar mobil bekas berdasarkan fitur penting seperti merek, tipe, tahun produksi, ukuran mesin, jarak tempuh, region, dan opsi kendaraan, sehingga dapat mendukung pengambilan keputusan bagi penjual, pembeli, dan platform secara objektif dan terukur. Keberhasilan model dievaluasi menggunakan metrik **Mean Absolute Error (MAE)**, dengan target awal MAE tidak melebihi 20% dari median harga sebagai indikator akurasi yang relevan secara bisnis.

Dataset **Saudi Arabia Used Cars** dari Kaggle yang berisi lebih dari 8.000 data mobil bekas dianalisis dan dipersiapkan melalui tahapan pemahaman struktur data, analisis statistik deskriptif, identifikasi *high-cardinality*, pemeriksaan *missing values* (terutama pada *Engine_Size*), serta analisis distribusi variabel numerik yang menunjukkan pola *right-skewed* dan keberadaan outlier signifikan. Proses pembersihan data mencakup imputasi konservatif, penghapusan kolom tidak relevan dan duplikat, serta penanganan outlier menggunakan metode **IQR**, diikuti dengan konversi tipe data dan *feature engineering* seperti **Car_Age** dan **Mileage_per_Year** untuk meningkatkan representasi informasi.

Tahap pemodelan dilakukan secara end-to-end dengan pembagian data **train–test 80:20** dan pembangunan **pipeline preprocessing** terintegrasi untuk mencegah data leakage, yang memisahkan perlakuan fitur numerik dan kategorikal. Beberapa algoritma regresi dievaluasi menggunakan **cross-validation** berbasis MAE, termasuk Linear Regression, Random Forest, Gradient Boosting, dan XGBoost. Model **XGBoost** menunjukkan performa terbaik dan paling stabil, dengan **MAE data uji sebesar ±11.200 SAR**, yang berada **di bawah 20% dari median harga kendaraan**, sehingga memenuhi target akurasi bisnis yang ditetapkan. Model kemudian ditingkatkan melalui **hyperparameter tuning (GridSearchCV)** yang menghasilkan penurunan MAE secara signifikan dibanding baseline.

Interpretabilitas model dianalisis menggunakan **SHAP**, yang mengidentifikasi *Engine_Size, Year, Mileage,* dan *Car_Age* sebagai faktor paling dominan dalam penentuan harga, diikuti oleh kontribusi merek dan tipe kendaraan tertentu. Model final disimpan sebagai artefak terlatih dan diimplementasikan dalam pipeline prediksi yang siap digunakan secara operasional, memungkinkan estimasi harga wajar pasar secara konsisten, terukur, dan berorientasi bisnis.


## 5.2 Limitasi

- Model ini memprediksi **harga yang tertera di iklan**, bukan harga akhir hasil negosiasi, sehingga perbedaan harga setelah tawar-menawar belum sepenuhnya tercermin.
- Informasi mengenai **kondisi teknis kendaraan** (misalnya kondisi mesin, riwayat servis, atau tingkat keausan) masih terbatas, sehingga kualitas mobil yang sebenarnya belum sepenuhnya terwakili dalam prediksi.
- Sebagian data didominasi oleh **merek, tahun, dan wilayah tertentu**, sehingga akurasi prediksi untuk mobil dengan karakteristik yang jarang muncul di data bisa lebih rendah.
- Model ini melihat **pola hubungan dari data historis**, tanpa mempertimbangkan perubahan harga dari waktu ke waktu atau faktor sebab-akibat tertentu.
- Model belum dilengkapi dengan **pemantauan performa dan pembaruan otomatis**, sehingga akurasi prediksi dapat menurun apabila kondisi pasar berubah.


## 5.3 Rekomendasi

- Menggabungkan **data harga transaksi sebenarnya dan riwayat negosiasi** agar prediksi semakin mendekati harga pasar yang benar-benar terjadi.
- Menambahkan **informasi kondisi kendaraan yang lebih detail**, seperti hasil inspeksi, riwayat servis, atau catatan kecelakaan, supaya kualitas mobil dapat tercermin lebih akurat dalam model.
- Membangun **model terpisah untuk segmen tertentu** (misalnya per merek, wilayah, atau rentang harga) agar performa tetap baik meskipun data tidak merata.
- Mengembangkan model yang **mempertimbangkan faktor waktu**, seperti tren pasar, inflasi, dan pola musiman, serta mulai mengeksplorasi hubungan sebab-akibat secara sederhana.
- Menerapkan **pemantauan performa model dan data secara berkala**, termasuk pembaruan model secara rutin, agar akurasi tetap terjaga setelah digunakan di lingkungan nyata.
